In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops

In [2]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
sess = tf.Session()
writer = tf.summary.FileWriter('/data/tensorflow/log', sess.graph)
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))
writer.close()

7.5
[ 3.  7.]


In [3]:
c = tf.greater([1.0,2.0,0.0,3.0,0.0, -.5], 0.0)

In [4]:
sess.run(c)

array([ True,  True, False,  True, False, False], dtype=bool)

In [5]:
x = tf.Variable(.5)
y = tf.cast(tf.greater(x, 0), tf.float32)
tf.gradients([y], [x])

[None]

In [6]:
sess.run(tf.multiply(tf.subtract(tf.cast(c, tf.float32), .5), 2))

array([ 1.,  1., -1.,  1., -1., -1.], dtype=float32)

In [7]:
@ops.RegisterGradient("DubIdent")
def dubIdent(op, grad):
    return [tf.ones(tf.shape(op.inputs[0])), tf.ones(tf.shape(op.inputs[1]))]

@ops.RegisterGradient("Bin_ST")
def bin_ST(op, grad):
    in_vals = op.inputs[0]
    grad_func = tf.cast(tf.less_equal(tf.abs(in_vals), 1), tf.float32)
    grad_out = tf.multiply(grad_func, grad)
    return [grad_out]

### binary sign function with straight through estimator gradient and associated gradient overrides ###
def binarySign(x):
    g = tf.get_default_graph()
    with ops.name_scope("binarySign") as name:       
        with g.gradient_override_map({"Sign": "Bin_ST"}):
            return tf.sign(x)

In [13]:
x = tf.constant([-1,2,-.3, .001, -.2])
y = binarySign(x)
out = tf.gradients([y], [x])

In [14]:
sess.run(out)

[array([ 1.,  0.,  1.,  1.,  1.], dtype=float32)]

In [10]:
g = tf.get_default_graph()
dir(g)
g.get_operations()

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'Greater/x' type=Const>,
 <tf.Operation 'Greater/y' type=Const>,
 <tf.Operation 'Greater' type=Greater>,
 <tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'Greater_1/y' type=Const>,
 <tf.Operation 'Greater_1' type=Greater>,
 <tf.Operation 'Cast' type=Cast>,
 <tf.Operation 'gradients/Shape' type=Const>,
 <tf.Operation 'gradients/Const' type=Const>,
 <tf.Operation 'gradients/Fill' type=Fill>,
 <tf.Operation 'Cast_1' type=Cast>,
 <tf.Operation 'Sub/y' type=Const>,
 <tf.Operation 'Sub' type=Sub>,
 <tf.Operation 'Mul/y' type=Const>,
 <tf.Operation 'Mul' type=Mul>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'binarySign/Sign' type=Sign>,
 <tf.Operation 'gradients_1/Shape' type=Const>,
 <tf